In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.utils import shuffle
import statsmodels.api as sm

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date', 'start_date_short', 'end_date_short'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [12]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/metro_rush.csv'
bikeshare_machine = set_data(file)

In [13]:
bikeshare_machine = bikeshare_machine.drop(['temperature', 'humidity', 'windspeed'], 1)
bikeshare_machine = bikeshare_machine.rename(columns = {'member_type_Registered':'member_type'})
bikeshare_machine.head()

,time_diff,miles,rush_hour,metro_dist,temp,hum,wind,member_type,holiday_1,work_day_1,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
56673,11.383,1.034046,0,0.309956,2.421733,42.3043,6.305571,1,0,1,...,0,0,0,1,0,0,0,1,0,0
75641,5.350,0.837833,0,0.586599,-1.215644,43.7391,14.869645,1,0,1,...,0,0,0,0,1,0,0,1,0,0
964828,7.950,1.154813,1,0.722611,9.977500,80.2917,12.124789,1,0,1,...,0,0,0,0,1,0,0,0,1,0
584802,8.083,0.869846,0,0.181761,25.370000,75.7500,13.209050,1,0,1,...,0,0,0,0,1,0,0,0,1,0
1083395,10.267,0.748221,0,0.048450,5.669151,39.5833,28.250014,1,0,0,...,0,0,0,0,0,0,1,1,0,0


### Model 1 - Remove Some Variables that are correlated

In [14]:
# Leave workday, drop weekdays, leave season, drop month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['weekday_0', 'weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6', 
              'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9',
              'month_10','month_11','month_12','time_diff']
X1 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y1 = bikeshare_machine['time_diff']

In [26]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((914885, 278), (304962, 278), (914885,), (304962,))

In [27]:
# Fit model
model1 = LinearRegression()
model1.fit(X1_train, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [28]:
pred1 = model1.predict(X1_test)
cvscores_model1 = cross_val_score(model1, X1_train, y1_train, cv=5)
model1_r = model1.score(X1_test, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [29]:
cvscores_model1, model1_r, adjustedr1, model1_mse, model1_rmse

(array([ -9.13376990e+05,   4.17575992e-01,   4.12811593e-01,
          4.12874687e-01,   2.96543058e-01]),
 0.41496608433864346,
 0.41443228550984812,
 80.493344138620685,
 8.9718082981426157)

In [30]:
X1_train

matrix([[1.4752076056302033, 0, 0.032515857047723216, ..., 1, 0, 0],
        [0.2772192968473278, 0, 0.3280928962753367, ..., 1, 0, 0],
        [0.0, 1, 0.06288958953434466, ..., 0, 1, 0],
        ..., 
        [0.8808793808702235, 0, 0.009423342924515923, ..., 1, 0, 0],
        [0.7052793368622997, 0, 0.12409417540342065, ..., 0, 0, 1],
        [2.661032613082074, 0, 0.639386065154096, ..., 1, 0, 0]], dtype=object)

In [32]:
model1_sm = sm.OLS(y1_train, X1_train.astype(float)).fit()
model1_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.415
Model:                            OLS   Adj. R-squared:                  0.415
Method:                 Least Squares   F-statistic:                     2380.
Date:                Tue, 12 Dec 2017   Prob (F-statistic):               0.00
Time:                        22:39:18   Log-Likelihood:            -3.3027e+06
No. Observations:              914885   AIC:                         6.606e+06
Df Residuals:                  914611   BIC:                         6.609e+06
Df Model:                         273                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             6.2122      0.013    463.198      0.000       6.186       6.239
x2            -1.4883      0.029    -50.680      0.000      -1.546      -1.431
x3             0.2323      0.184      1.263      0.207      -0.128       0.593
x4             0.0559      0.002     28.420      0.000       0.052       0.060
x5             0.0006      0.001      0.601      0.548      -0.001       0.002
x6            -0.0051      0.002     -2.527      0.012      -0.009      -0.001
x7           -11.2421      0.028   -408.483      0.000     -11.296     -11.188
x8            -0.5302      0.064     -8.235      0.000      -0.656      -0.404
x9            -1.4145      0.023    -62.044      0.000      -1.459      -1.370
x10           -0.5345      0.220     -2.425      0.015      -0.966      -0.102
x11           -1.0267      0.128     -8.001      0.000      -1.278      -0.775
x12            3.5067      0.124     28.341      0.000       3.264       3.749
x13           -1.2004      0.253     -4.745      0.000      -1.696      -0.705
x14           -1.7809      0.119    -14.979      0.000      -2.014      -1.548
x15            1.2862      0.306      4.209      0.000       0.687       1.885
x16           -0.1700      0.324     -0.525      0.600      -0.805       0.465
x17           -1.5921      0.138    -11.545      0.000      -1.862      -1.322
x18           -1.0375      0.157     -6.604      0.000      -1.345      -0.730
x19            0.2092      0.126      1.662      0.097      -0.038       0.456
x20            2.9997      0.137     21.962      0.000       2.732       3.267
x21           -0.8645      0.141     -6.142      0.000      -1.140      -0.589
x22            1.1264      0.129      8.745      0.000       0.874       1.379
x23           -2.2054      0.119    -18.496      0.000      -2.439      -1.972
x24           -1.2861      0.114    -11.318      0.000      -1.509      -1.063
x25           -0.8132      0.108     -7.499      0.000      -1.026      -0.601
x26           -1.2722      0.110    -11.604      0.000      -1.487      -1.057
x27           -2.1147      0.140    -15.098      0.000      -2.389      -1.840
x28           -1.6844      0.236     -7.140      0.000      -2.147      -1.222
x29            3.0774      0.233     13.189      0.000       2.620       3.535
x30           -0.4811      0.102     -4.735      0.000      -0.680      -0.282
x31           -1.7111      0.281     -6.084      0.000      -2.262      -1.160
x32           -2.0032      0.111    -18.084      0.000      -2.220      -1.786
x33           -0.5910      0.102     -5.799      0.000      -0.791      -0.391
x34            0.0721      0.137      0.526      0.599      -0.197       0.341
x35           -0.4841      0.120     -4.041      0.000      -0.719      -0.249
x36           -0.0349      0.210     -0.166      0.868      -0.447       0.378
x3

### Model 2 - Try reverse of Model 1

In [34]:
# Drop workday, leave weekdays, drop season, leave month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['work_day_1','season_1', 'season_2', 'season_3', 'season_4','time_diff']
X2 = bikeshare_machine.drop(remove_cols, 1)
y2 = bikeshare_machine['time_diff']

In [35]:
# Split the data into training and testing sets and check the shape
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.25)
X2_train.shape, X2_test.shape, y2_train.shape, y2_test.shape

((914885, 292), (304962, 292), (914885,), (304962,))

In [36]:
# Fit model
model2 = LinearRegression()
model2.fit(X2_train, y2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [37]:
pred2 = model2.predict(X2_test)
cvscores_model2 = cross_val_score(model2, X2_train, y2_train, cv=5)
model2_r = model2.score(X2_test, y2_test)
model2_mse = mean_squared_error(y2_test, pred2)
model2_rmse = np.sqrt(model2_mse)
adjustedr2 = 1 - (1-model2_r)*(len(y2_test)-1)/(len(y2_test)-X2_test.shape[1]-1)

In [38]:
cvscores_model2, model2_r, adjustedr2, model2_mse, model2_rmse

(array([  4.14856314e-01,   4.16613112e-01,  -8.46136519e+11,
          4.15043429e-01,   4.19197047e-01]),
 0.41181867887796764,
 0.41125495580221128,
 80.276327546184859,
 8.9597057734160472)

In [45]:
# Use statsmodel to check the statistical side of the model
model2_sm = sm.OLS(y2_train, np.matrix(X2_train).astype(float)).fit()
model2_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.417
Model:                            OLS   Adj. R-squared:                  0.417
Method:                 Least Squares   F-statistic:                     2290.
Date:                Tue, 12 Dec 2017   Prob (F-statistic):               0.00
Time:                        23:03:08   Log-Likelihood:            -3.3024e+06
No. Observations:              914885   AIC:                         6.605e+06
Df Residuals:                  914598   BIC:                         6.609e+06
Df Model:                         286                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             6.2107      0.013    463.406      0.000       6.184       6.237
x2            -1.4635      0.029    -49.815      0.000      -1.521      -1.406
x3             0.1504      0.190      0.792      0.428      -0.222       0.523
x4             0.0605      0.003     21.277      0.000       0.055       0.066
x5            -0.0045      0.001     -4.436      0.000      -0.006      -0.002
x6            -0.0171      0.002     -8.236      0.000      -0.021      -0.013
x7           -11.2659      0.028   -409.447      0.000     -11.320     -11.212
x8             0.6540      0.066      9.872      0.000       0.524       0.784
x9            -0.4033      0.223     -1.812      0.070      -0.840       0.033
x10           -1.1239      0.130     -8.677      0.000      -1.378      -0.870
x11            3.3154      0.124     26.637      0.000       3.071       3.559
x12           -1.0330      0.255     -4.048      0.000      -1.533      -0.533
x13           -1.8199      0.119    -15.279      0.000      -2.053      -1.586
x14            1.3689      0.311      4.408      0.000       0.760       1.978
x15           -0.3977      0.322     -1.236      0.216      -1.028       0.233
x16           -1.5380      0.139    -11.034      0.000      -1.811      -1.265
x17           -1.0456      0.159     -6.561      0.000      -1.358      -0.733
x18            0.1507      0.127      1.188      0.235      -0.098       0.399
x19            2.9290      0.137     21.357      0.000       2.660       3.198
x20           -0.8835      0.141     -6.267      0.000      -1.160      -0.607
x21            1.0220      0.129      7.901      0.000       0.768       1.276
x22           -2.4042      0.120    -20.008      0.000      -2.640      -2.169
x23           -1.3470      0.114    -11.830      0.000      -1.570      -1.124
x24           -0.8599      0.109     -7.896      0.000      -1.073      -0.646
x25           -1.4366      0.110    -13.022      0.000      -1.653      -1.220
x26           -2.1965      0.141    -15.588      0.000      -2.473      -1.920
x27           -1.8828      0.239     -7.880      0.000      -2.351      -1.414
x28            2.7162      0.236     11.510      0.000       2.254       3.179
x29           -0.5753      0.102     -5.637      0.000      -0.775      -0.375
x30           -1.8614      0.281     -6.617      0.000      -2.413      -1.310
x31           -2.1572      0.111    -19.401      0.000      -2.375      -1.939
x32           -0.6816      0.102     -6.666      0.000      -0.882      -0.481
x33           -0.0512      0.139     -0.369      0.712      -0.324       0.221
x34           -0.6356      0.121     -5.263      0.000      -0.872      -0.399
x35           -0.3482      0.212     -1.639      0.101      -0.764       0.068
x36            0.6215      0.147      4.235      0.000       0.334       0.909
x3

### Model 3 - All variables

In [46]:
# Keep all but time_diff
X3 = np.matrix(bikeshare_machine.drop('time_diff', 1))
y3 = bikeshare_machine['time_diff']

In [52]:
# Split the data into training and testing sets and check the shape
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size = 0.25)
X3_train.shape, X3_test.shape, y3_train.shape, y3_test.shape

((914885, 297), (304962, 297), (914885,), (304962,))

In [53]:
# Fit model
model3 = LinearRegression()
model3.fit(X3_train, y3_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [54]:
pred3 = model3.predict(X3_test)
cvscores_model3 = cross_val_score(model3, X3_train, y3_train, cv=5)
model3_r = model3.score(X3_test, y3_test)
model3_mse = mean_squared_error(y3_test, pred3)
model3_rmse = np.sqrt(model3_mse)
adjustedr3 = 1 - (1-model3_r)*(len(y3_test)-1)/(len(y3_test)-X3_test.shape[1]-1)

In [55]:
cvscores_model3, model3_r, adjustedr3, model3_mse, model3_rmse

(array([  4.16048247e-01,  -1.09361915e+10,   4.13925593e-01,
          4.20226697e-01,   4.13314359e-01]),
 0.41415063863003854,
 0.4135795266498673,
 80.26369032135203,
 8.95900052022278)

In [57]:
# Use statsmodel to check the statistical side of the model
model3_sm = sm.OLS(y3_train, X3_train.astype(float)).fit()
model3_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.416
Method:                 Least Squares   F-statistic:                     2259.
Date:                Wed, 13 Dec 2017   Prob (F-statistic):               0.00
Time:                        19:09:42   Log-Likelihood:            -3.3024e+06
No. Observations:              914885   AIC:                         6.605e+06
Df Residuals:                  914595   BIC:                         6.609e+06
Df Model:                         289                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             6.2249      0.013    464.213      0.000       6.199       6.251
x2            -1.4812      0.029    -50.413      0.000      -1.539      -1.424
x3            -0.0281      0.191     -0.147      0.883      -0.402       0.346
x4             0.0598      0.003     21.016      0.000       0.054       0.065
x5            -0.0045      0.001     -4.472      0.000      -0.006      -0.003
x6            -0.0156      0.002     -7.457      0.000      -0.020      -0.011
x7           -11.2329      0.028   -408.412      0.000     -11.287     -11.179
x8             3.5788      0.059     61.140      0.000       3.464       3.694
x9             2.9497      0.039     75.759      0.000       2.873       3.026
x10           -0.6154      0.221     -2.788      0.005      -1.048      -0.183
x11           -1.3148      0.129    -10.199      0.000      -1.567      -1.062
x12            3.2616      0.124     26.248      0.000       3.018       3.505
x13           -1.0278      0.255     -4.026      0.000      -1.528      -0.527
x14           -1.8745      0.119    -15.745      0.000      -2.108      -1.641
x15            0.8450      0.311      2.713      0.007       0.234       1.456
x16           -0.4365      0.323     -1.350      0.177      -1.070       0.197
x17           -1.6432      0.140    -11.715      0.000      -1.918      -1.368
x18           -1.0783      0.161     -6.711      0.000      -1.393      -0.763
x19           -0.0510      0.126     -0.404      0.686      -0.299       0.197
x20            2.7964      0.137     20.412      0.000       2.528       3.065
x21           -0.9942      0.141     -7.044      0.000      -1.271      -0.718
x22            0.8164      0.129      6.329      0.000       0.564       1.069
x23           -2.4644      0.120    -20.608      0.000      -2.699      -2.230
x24           -1.3724      0.114    -12.019      0.000      -1.596      -1.149
x25           -0.9396      0.110     -8.576      0.000      -1.154      -0.725
x26           -1.5021      0.110    -13.646      0.000      -1.718      -1.286
x27           -2.1427      0.141    -15.172      0.000      -2.420      -1.866
x28           -1.7858      0.241     -7.407      0.000      -2.258      -1.313
x29            2.7503      0.234     11.770      0.000       2.292       3.208
x30           -0.5403      0.103     -5.261      0.000      -0.742      -0.339
x31           -1.8591      0.279     -6.667      0.000      -2.406      -1.313
x32           -2.2305      0.111    -20.088      0.000      -2.448      -2.013
x33           -0.8009      0.102     -7.820      0.000      -1.002      -0.600
x34           -0.0518      0.138     -0.374      0.708      -0.323       0.219
x35           -0.7782      0.120     -6.466      0.000      -1.014      -0.542
x36           -0.3872      0.213     -1.819      0.069      -0.804       0.030
x3

### Model 4 - Ridge with Round 1

In [58]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge1 = Ridge(fit_intercept=True)
ridge1_gs = GridSearchCV(ridge1, param_grid, cv=5)
ridge1_gs.fit(X1_train, y1_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [59]:
pred4 = ridge1_gs.predict(X1_test)
ridge1_r = ridge1_gs.score(X1_test, y1_test)
ridge1_mse = mean_squared_error(y1_test, pred4)
ridge1_rmse = np.sqrt(ridge1_mse)
adjustedr4 = 1 - (1-ridge1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [60]:
ridge1_r, ridge1_mse, ridge1_rmse, adjustedr4

(0.41496608576252131,
 80.493343942712926,
 8.9718082872246505,
 0.4144322869350251)

### Ridge with Round 2

In [61]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge2 = Ridge(fit_intercept=True)
ridge2_gs = GridSearchCV(ridge2, param_grid, cv=5)
ridge2_gs.fit(X2_train, y2_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [62]:
pred5 = ridge2_gs.predict(X2_test)
ridge2_r = ridge2_gs.score(X2_test, y2_test)
ridge2_mse = mean_squared_error(y2_test, pred5)
ridge2_rmse = np.sqrt(ridge2_mse)
adjustedr5 = 1 - (1-ridge2_r)*(len(y2_test)-1)/(len(y2_test)-X2_test.shape[1]-1)

In [63]:
ridge2_r, ridge2_mse, ridge2_rmse, adjustedr5

(0.41181867087249791,
 80.276328638789622,
 8.9597058343892986,
 0.4112549477890689)

### Ridge with all data

In [64]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge3 = Ridge(fit_intercept=True)
ridge3_gs = GridSearchCV(ridge3, param_grid, cv=5)
ridge3_gs.fit(X3_train, y3_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [65]:
pred6 = ridge3_gs.predict(X3_test)
ridge3_r = ridge3_gs.score(X3_test, y3_test)
ridge3_mse = mean_squared_error(y3_test, pred6)
ridge3_rmse = np.sqrt(ridge3_mse)
adjustedr6 = 1 - (1-ridge3_r)*(len(y3_test)-1)/(len(y3_test)-X3_test.shape[1]-1)

In [66]:
ridge3_r, ridge3_mse, ridge3_rmse, adjustedr6

(0.41415064749101299,
 80.263689107363319,
 8.9590004524703151,
 0.4135795355194799)

In [39]:
#bikeshare_machine.to_csv('machine_full.csv')